### Read Excel file, extract URLs from column A into a list, and treat first line as a header

https://colab.research.google.com/drive/1f_1HeD1mK_wXfjgvY4VGNFKSQBE5Imeh?usp=sharing#scrollTo=jm2nTHTtVTp0

In [36]:
import sys
import pandas as pd
sys.path.append('../..')
from py3810.myUtils import pickle_dump, pickle_load

# Set the path to the directory containing the Excel file
path_lumen_dump = "../langchain/docs/lumen/"
path_lumen_docs = path_lumen_dump + "docs/"

lumen_urls = pickle_load(filename_pickle='lumen_urls', path_pickle_dump=path_lumen_dump)

In [30]:
from langchain.document_loaders import WebBaseLoader
from langchain_community.document_transformers import Html2TextTransformer

loader = WebBaseLoader(lumen_urls)
docs_html = loader.load()
html2text = Html2TextTransformer()
docs_text = html2text.transform_documents(docs_html)

In [31]:
import re

def remove_newlines(text):
  # r'\n|\s{2,}' finds both newlines (\n) and multiple whitespaces (\s{2,}) 
  # and replace them with " ", .strip() strips leading and trailing spaces 
  return re.sub(r'\n|\s{2,}', ' ', text).strip()

text = "   \n  This is a string  \n  with   multiple \n \n \nnewlines.  "
text_cleaned = remove_newlines(text)
print(f'text: {text}')
print('=========')
print(f'text_cleaned: {text_cleaned}')

text:    
  This is a string  
  with   multiple 
 
 
newlines.  
text_cleaned: This is a string with multiple newlines.


In [ ]:
text_lumen_url_footer = \
'SCHEDULE AN APPOINTMENTLumen Optometric 14 West Sierra Madre Blvd, \
Sierra Madre, CA 91024 (626) 921-0199 info@lumenoptometric.comServicesComprehensive \
Eye Exams Contact Lens Exams Orthokeratology Neurolens Therapy Scleral Lenses / Keratoconus \
Quick LinksHome About Us Sitemap Hours Of OperationTuesday9:45 am - 5:30 \
pmWednesday9:45 am - 5:30 pmThursday9:45 am - 1:30 pmFriday9:45 am - 5:30 \
pmSaturday9:45 am - 5:30 pmCopyright © 2020 Lumen Optometric. All Rights Reserved.\
X[contact-form-7 404 "Not Found"]Top'

for i, doc in enumerate(docs_text):
  doc.page_content = remove_newlines(doc.page_content)
  if text_lumen_url_footer in doc.page_content:
    doc.page_content = doc.page_content.replace(text_lumen_url_footer, "")
    print(f'docs_text[{i}]: remove text_lumen_url_footer')
    doc.page_content = remove_newlines(doc.page_content)

https://platform.openai.com/docs/tutorials/web-qa-embeddings

In [33]:
from langchain.docstore.document import Document

In [34]:
lumen_office_info = \
  "Lumen Optometric address is located at 14 West Sierra Madre Blvd, Sierra Madre, CA 91024. \
   Lumen Optometric office is located at 14 West Sierra Madre Blvd, Sierra Madre, CA 91024. \
   Lumen Optometric location is 14 West Sierra Madre Blvd, Sierra Madre, CA 91024. \
   Lumen Optometric phone numbers are (626) 921-0199, (626) 507-2724. \
   Lumen Optometric office hours are Tuesday, Wednesday, Friday, and Saturday from 9:45 am to 5:30 pm, \
   and Thursday from 9:45 am to 1:30 pm. \
   Lumen Optometric email is info@lumenoptometric.com \
   Lumen Optometric website url is www.lumenoptometric.com"

doc_office_info = [Document(
  page_content=lumen_office_info,
  metadata={"description": "info, address, office location, phone number, office hours, email, url"}
  )]

In [35]:
docs_text = doc_office_info + docs_text

In [38]:
pickle_dump(file_to_pickle=docs_text, filename_pickle='lumen_docs_website', path_pickle_dump=path_lumen_docs)

In [37]:
# docs = pickle_load(filename_pickle='lumen_docs_website', path_pickle_dump=path_lumen_docs)